In [17]:
import os
import mido
from mido import MidiFile, MidiTrack, Message
import random

import json

import geneticalgorithm as ga

In [3]:
def generate_random(snote=50, mlength=12, numofmidi=10, time=150, filename='random', pitchrnd=False):
    
    notes = range(snote, snote+mlength)
    
    noterange = range(mlength)

    # pitch range for random pitch value ;
    pitches = range(-8192,8191)
    
    # Create music folder if it does not exist
    if not os.path.exists('music'):
        os.makedirs('music')

    for j in range(numofmidi):
    
        mid = MidiFile(type=0) # type0 can have only one track
    
        track = MidiTrack() # note list (kind of)

        mid.tracks.append(track)
    
        # the note which the pitch will change for
        pitchnote = random.choice(noterange)
        numofpnote = random.choice(noterange)
    
        for i in noterange:
        
            note = random.choice(notes)
            pitch = random.choice(pitches)
        
            if pitchrnd:
                if i == pitchnote: # Change the pitch on the note
                    track.append(Message('pitchwheel', pitch=pitch))
                if i == (pitchnote+numofpnote): # Change the pitch back to default
                    track.append(Message('pitchwheel'))
        
            track.append(Message('note_on', note=note, time=time))
            track.append(Message('note_off', note=note, time=time))
            
        note = random.choice(notes)
        track.append(Message('note_on', note=note, time=time))
        track.append(Message('note_off', note=note, time=500))
        

        mid.save('music/' + filename + str(j) + '.mid')

In [4]:
def apply_mutation(mutantnotelist, midino, snote=50, time=150, filename='random'):

    mid = MidiFile(type=0) # type0 can have only one track
    
    track = MidiTrack() # note list (kind of)

    mid.tracks.append(track)
    
    # Create mutant music folder if it does not exist
    if not os.path.exists('mutantmusic'):
        os.makedirs('mutantmusic')
    
    # add the octaves back
    mutantnotelist2 = [x+snote for x in mutantnotelist]
    
    for note in mutantnotelist2[:10]:
        
        #print(note)
        
        track.append(Message('note_on', note=int(note), time=time))
        track.append(Message('note_off', note=int(note), time=time))
        
    track.append(Message('note_on', note=mutantnotelist2[11], time=time))
    track.append(Message('note_off', note=mutantnotelist2[11], time=500))
        
        
    mid.save('mutantmusic/' + filename + str(midino) + '.mid')

In [5]:
def read_midi(midiname, snote=50):
    
    mid = MidiFile(midiname)
    
    noteonlist = []
    for i, track in enumerate(mid.tracks):
        #print('Track {}: {}'.format(i, track.name))
        for message in track:
            #print(message)
            if message.type == 'note_on':
                noteonlist.append(message.note)
    
    # normalize the note integers for mutation by reducing octaves
    notelist = [x-snote for x in noteonlist]
    
    return notelist

In [6]:
mlength = 11
snote = 50 # starting note
numofmidi = 10 # number of midi files
time = 150
filename='random'
pitchrnd=False # do not include pitch variations for now

In [6]:
generate_random(snote, mlength, numofmidi, time, filename, pitchrnd)

for j in range(numofmidi):
    
    midiname = 'music/' + filename + str(j) + '.mid'
    
    notelist = read_midi(midiname)
    
    mutantnotelist = ga.mutate(notelist, mlength)
    
    apply_mutation(mutantnotelist, j, snote, time, filename)
    
    

In [27]:
notelists = {}

In [28]:
# Read the notes and generate the integer list 
for j in range(10):
    
    for filename in ['gf', 'gs', 'r']:
    
        midiname = 'melodies/' + filename + str(j) + '.mid'
    
        notelist = read_midi(midiname)
    
        print(filename, str(j), notelist)
    
        notelists[filename + str(j)] = notelist
    

print(notelists)

gf 0 [11, 9, 7, 4, 2, 2, 2, 2, 4, 2, 2, 9]
gs 0 [3, 9, 9, 9, 1, 11, 9, 7, 7, 7, 7, 10]
r 0 [7, 2, 4, 5, 10, 11, 2, 4, 7, 5, 5, 7, 4]
gf 1 [11, 9, 7, 6, 4, 2, 0, 11, 9, 7, 7, 4]
gs 1 [5, 3, 2, 0, 11, 9, 7, 9, 10, 0, 10, 5]
r 1 [11, 3, 11, 10, 2, 0, 2, 9, 7, 11, 0, 0, 2]
gf 2 [11, 9, 7, 6, 4, 2, 11, 9, 7, 7, 9, 3]
gs 2 [3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10]
r 2 [10, 9, 9, 8, 1, 0, 7, 5, 4, 5, 4, 11, 5]
gf 3 [1, 11, 9, 11, 9, 9, 7, 6, 4, 2, 11, 11]
gs 3 [8, 0, 10, 9, 7, 5, 0, 0, 0, 0, 0, 2]
r 3 [3, 10, 10, 11, 4, 1, 2, 0, 1, 7, 7, 7, 1]
gf 4 [11, 9, 7, 6, 4, 2, 2, 0, 11, 9, 9, 5]
gs 4 [8, 6, 4, 2, 0, 11, 9, 7, 9, 1, 11, 9]
r 4 [5, 4, 1, 1, 0, 9, 7, 0, 0, 9, 11, 9, 2]
gf 5 [1, 11, 9, 7, 9, 11, 9, 11, 9, 11, 9, 11]
gs 5 [10, 1, 0, 10, 9, 7, 7, 7, 7, 7, 7, 7]
r 5 [0, 7, 1, 1, 3, 6, 9, 1, 7, 11, 8, 6, 6]
gf 6 [4, 2, 0, 11, 9, 7, 6, 4, 2, 0, 11, 5]
gs 6 [8, 6, 4, 2, 8, 0, 10, 9, 7, 9, 10, 9]
r 6 [5, 7, 4, 7, 4, 0, 11, 9, 2, 2, 7, 7, 1]
gf 7 [11, 9, 11, 9, 9, 11, 9, 9, 7, 7, 9, 6]
gs 7 [5, 4, 2,

In [36]:
with open('notelists.json') as f:    
    data = json.load(f)

In [38]:
data

{'gf0': [11, 9, 7, 4, 2, 2, 2, 2, 4, 2, 2, 9],
 'gf1': [11, 9, 7, 6, 4, 2, 0, 11, 9, 7, 7, 4],
 'gf2': [11, 9, 7, 6, 4, 2, 11, 9, 7, 7, 9, 3],
 'gf3': [1, 11, 9, 11, 9, 9, 7, 6, 4, 2, 11, 11],
 'gf4': [11, 9, 7, 6, 4, 2, 2, 0, 11, 9, 9, 5],
 'gf5': [1, 11, 9, 7, 9, 11, 9, 11, 9, 11, 9, 11],
 'gf6': [4, 2, 0, 11, 9, 7, 6, 4, 2, 0, 11, 5],
 'gf7': [11, 9, 11, 9, 9, 11, 9, 9, 7, 7, 9, 6],
 'gf8': [4, 2, 2, 2, 0, 11, 9, 9, 7, 6, 4, 10],
 'gf9': [1, 11, 9, 11, 9, 11, 9, 11, 9, 1, 11, 11],
 'gs0': [3, 9, 9, 9, 1, 11, 9, 7, 7, 7, 7, 10],
 'gs1': [5, 3, 2, 0, 11, 9, 7, 9, 10, 0, 10, 5],
 'gs2': [3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10],
 'gs3': [8, 0, 10, 9, 7, 5, 0, 0, 0, 0, 0, 2],
 'gs4': [8, 6, 4, 2, 0, 11, 9, 7, 9, 1, 11, 9],
 'gs5': [10, 1, 0, 10, 9, 7, 7, 7, 7, 7, 7, 7],
 'gs6': [8, 6, 4, 2, 8, 0, 10, 9, 7, 9, 10, 9],
 'gs7': [5, 4, 2, 2, 2, 2, 3, 2, 0, 11, 9, 0],
 'gs8': [10, 8, 7, 5, 4, 2, 0, 11, 9, 7, 6, 1],
 'gs9': [0, 10, 9, 7, 6, 4, 6, 1, 4, 1, 11, 5],
 'r0': [7, 2, 4, 5, 10, 11, 2, 4,

In [40]:
len(data)

30